In [1]:
import requests
import pandas as pd
import datetime
from datetime import datetime
import json
import mysql.connector


## Variables

In [20]:
eject = True
all_ids = []
df_customval = pd.DataFrame()
j_cvalue_all = pd.DataFrame()
#j_cvalue_all = []

### List of all Contacts ###

In [21]:
## API Connection Contacts

url_contact = "https://{link}.api-us1.com/api/3/contacts?limit=100"

headers = {
    "accept": "application/json",
    "Api-Token": "{Your API TOken}",
    'limit': str(100),    # número máximo de resultados por página
    'offset': str(0)    # número de resultados para omitir al principio
    
}
while eject == True:
    
    response_contact = requests.get(url = url_contact, headers=headers)
    
    if response_contact.status_code == 200:
        jcontact = response_contact.json()
        df_contacts = pd.json_normalize(jcontact['contacts'])
        jcontact
        print('Connetion successful ')

        
        current_offset = headers['offset']
        total_contacts = len(jcontact['contacts'])
        next_offset = str(int(current_offset) + int(total_contacts) )
        
        if next_offset < jcontact['meta']['total']:
            headers["offset"] = str( next_offset)
            
        elif headers['offset'] != jcontact['meta']['total']:
            headers["offset"] = str( jcontact['meta']['total'])
        else:
            eject = False
        
        all_ids.extend(df_contacts['id'])
    
    else:
        print("Connection Error: " + str(response_contact.status_code))

Connetion successful 
Connetion successful 
Connetion successful 
Connetion successful 
Connetion successful 
Connetion successful 
Connetion successful 


## Request of the Custom Values from a Specific Contact

In [22]:

for i in all_ids:
    
    url_cvalues = "https://{link}.api-us1.com/api/3/contacts/" + str(i) + "/fieldValues"

    headers = {
        "accept": "application/json",
        "Api-Token": "{Your Api Token}"
    }
        
    response_cvalues = requests.get(url_cvalues, headers=headers)
    
    if response_cvalues.status_code == 200: 
    
        response_cvalues.text
        jcvalues = response_cvalues.json()
        j_cvalues = pd.json_normalize(jcvalues['fieldValues'])
        j_cvalue_all = pd.concat([j_cvalue_all, j_cvalues])
                        
    else: 
        print("Connection Error: " + str(response_cvalues.status_code))
        
j_cvalue_all = j_cvalue_all[['id','contact','field','value','cdate','udate']]
 

In [24]:
j_cvalue_all

,id,contact,field,value,cdate,udate
0,229,451,31,Operator,2023-02-21T13:45:26-06:00,2023-02-21T13:45:26-06:00
1,230,451,32,Matchmaker,2023-02-21T13:45:26-06:00,2023-02-21T13:45:26-06:00
2,231,451,34,Cold Lead,2023-02-21T13:45:26-06:00,2023-02-21T13:45:26-06:00
0,153,424,31,Operator,2023-02-21T13:45:23-06:00,2023-02-21T13:45:23-06:00
1,154,424,32,Facebook Group,2023-02-21T13:45:23-06:00,2023-02-21T13:45:23-06:00
...,...,...,...,...,...,...
23,1473,889,25,,2023-03-02T10:57:01-06:00,2023-03-02T10:57:01-06:00
24,1474,889,26,,2023-03-02T10:57:01-06:00,2023-03-02T10:57:01-06:00
25,1475,889,27,,2023-03-02T10:57:01-06:00,2023-03-02T10:57:01-06:00
26,1476,889,28,,2023-03-02T10:57:01-06:00,2023-03-02T10:57:01-06:00


## List of all the Custom Fields

In [23]:
url = "https://{link}.api-us1.com/api/3/fields?limit=100"

headers = {
    "accept": "application/json",
    "Api-Token": "{Your Api Token}"
}

response_cfield = requests.get(url, headers=headers)
if response_cfield.status_code == 200: 
    j_cfield = response_cfield.json()
    j_cfield = pd.json_normalize(j_cfield['fields'])
    j_cfield = j_cfield[['id','title','perstag','type','udate','created_timestamp','updated_timestamp']]
else: 
    print("Connection Error: " + str(response_cfield.status_code))

## INSERT INTO Database

In [1]:
mydb = mysql.connector.connect(
  host="{Host}",
  user="{user}",
  password="{password}",
  database = "{database}"
)


NameError: name 'mysql' is not defined

In [35]:
inset_cfield = "INSERT into activecampaign_customfield (Id,Title,Perstag,Type,Udate,Cdate,UpdateTimestamp) VALUES (%s,%s,%s,%s,%s,%s,%s)"
insert_cvalue = "INSERT into activecampaign_contactcustomvalue (Id,IdContact,IdCustomField,Value,Cdate,Udate) VALUES (%s,%s,%s,%s,%s,%s)"

cfieldsql = ','.join(['%s' for _ in range(len(j_cfield.columns))])
cfieldsql = ','.join(['%s' for _ in range(len(j_cfield.columns))])

In [40]:

if mydb.is_connected():
    print("Connection succesfully")
    
    
    cursor = mydb.cursor()
    
    datavalue = [tuple(x) for x in j_cvalue_all.to_numpy()]
    datafield = [tuple(x) for x in j_cfield.to_numpy()]

    # Ejecutar el comando SQL
    cursor.executemany(insert_cvalue, datavalue)
    cursor.executemany(inset_cfield, datafield)
    # Commit Insert
    mydb.commit()

    print(j_cvalue_all)
    cursor.close()
    print("DB closed: " + str(cursor))
    
else: 
    print("Connection Error: " + str(mydb.is_connected()))  

#Close Connection

#Close Database Connection
mydb.close()
print("DB closed: " + str(mydb))



Connection succesfully
      id contact field           value                      cdate  \
0    229     451    31        Operator  2023-02-21T13:45:26-06:00   
1    230     451    32      Matchmaker  2023-02-21T13:45:26-06:00   
2    231     451    34       Cold Lead  2023-02-21T13:45:26-06:00   
0    153     424    31        Operator  2023-02-21T13:45:23-06:00   
1    154     424    32  Facebook Group  2023-02-21T13:45:23-06:00   
..   ...     ...   ...             ...                        ...   
23  1473     889    25                  2023-03-02T10:57:01-06:00   
24  1474     889    26                  2023-03-02T10:57:01-06:00   
25  1475     889    27                  2023-03-02T10:57:01-06:00   
26  1476     889    28                  2023-03-02T10:57:01-06:00   
27  1477     889    29                  2023-03-02T10:57:01-06:00   

                        udate  
0   2023-02-21T13:45:26-06:00  
1   2023-02-21T13:45:26-06:00  
2   2023-02-21T13:45:26-06:00  
0   2023-02-21T13:45